In [67]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
#models:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor,\
GradientBoostingRegressor, VotingRegressor, BaggingRegressor, ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
##
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
import seaborn as sns
import pickle
%matplotlib inline

In [2]:
# Read the train and test datasets
df_train = pd.read_csv("dataset/train_final.csv")
df_test = pd.read_csv("dataset/test_final.csv")

cols_train = df_train.columns.tolist()
cols_test = df_test.columns.tolist()

# Train the model with columns that exist both in train and test set
cols_to_train = [col for col in cols_train if col in cols_test]
cols_to_train.remove('Id')

In [3]:
X_train_i = df_train[cols_to_train]
Y_train = df_train['SalePrice']
X_test_i = df_test[cols_to_train]
X_Id = df_test['Id']

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_i)
X_test = scaler.fit_transform(X_test_i)

In [5]:
print("Training(+validation) set shape : {}".format(X_train.shape))
print("Y_train shape : {}".format(Y_train.shape))
print("Test set shape : {}".format(X_test.shape))

Training(+validation) set shape : (1460, 160)
Y_train shape : (1460,)
Test set shape : (1459, 160)


In [76]:
LogisticRegression?

In [102]:
MLA = [
    LinearRegression(),
    LogisticRegression(solver='sag',max_iter=300),
    GaussianNB(),
#     MLPRegressor(max_iter=1000,verbose=1),
    GradientBoostingRegressor(),
#     VotingRegressor(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)]),
    BaggingRegressor(),
    ExtraTreesRegressor(),
    DecisionTreeRegressor(),
#     SVC(probability=True),
    KNeighborsRegressor(n_neighbors = 4),
    RandomForestRegressor(n_estimators = 100)
]

In [103]:
#split dataset in cross-validation with this splitter class: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit
#note: this is an alternative to train_test_split
cv_split = ShuffleSplit(n_splits = 10, test_size = .20, train_size = .80, \
                                                random_state = 0 )
                    # run model 10x with 60/30 split intentionally leaving out 10%

#create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Error Mean', 'MLA Test Error Mean', \
               'MLA Test Error 3*STD' ,'MLA Time']
MLA_compare = pd.DataFrame(columns = MLA_columns)

#create table to compare MLA predictions
MLA_predict = Y_train.copy()

#index through MLA and save performance to table
row_index = 0
for alg in MLA:

    #set name and parameters
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    print("Running the classification on %s" %(MLA_name))
    
    #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
    cv_results = cv_results = cross_validate(alg, X_train, Y_train, cv = cv_split,return_train_score=\
                                             True, scoring='neg_mean_absolute_error')

    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Train Error Mean'] = cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test Error Mean'] = cv_results['test_score'].mean()   
    #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, 
    #should statistically capture 99.7% of the subsets
    MLA_compare.loc[row_index, 'MLA Test Error 3*STD'] = cv_results['test_score'].std()*3   
    #let's know the worst that can happen!
    

    #save MLA predictions - see section 6 for usage
    alg.fit(X_train, Y_train)
    MLA_predict[MLA_name] = alg.predict(X_train)
    
    row_index+=1

    
#print and sort table: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
MLA_compare.sort_values(by = ['MLA Test Error Mean'], ascending = False, inplace = True)
MLA_compare
#MLA_predict

Running the classification on LinearRegression
{'fit_time': array([0.01097035, 0.00897479, 0.00797987, 0.00897694, 0.0079782 ,
       0.00797749, 0.00997114, 0.01196909, 0.01296329, 0.01296592]), 'score_time': array([0.00099826, 0.00099826, 0.00099707, 0.0009973 , 0.00099683,
       0.00099754, 0.        , 0.        , 0.00099802, 0.00099993]), 'test_score': array([-8.61847856e+13, -2.18221548e+04, -2.33768255e+04, -4.92749520e+13,
       -2.06098144e+04, -2.19351161e+04, -2.27909121e+04, -1.39296591e+14,
       -2.02047065e+04, -1.11932944e+13]), 'train_score': array([-15274.84837864, -17385.16670576, -15312.38789964, -15904.54728636,
       -16288.71634213, -16892.93276407, -16646.92582743, -16995.36856539,
       -16001.00502997, -17771.3098897 ])}
Running the classification on LogisticRegression


D:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


{'fit_time': array([ 95.93451691,  97.30186224,  94.15225458, 101.05682349,
       100.54619074,  93.19985533,  96.25066757,  98.38396788,
        95.47180581,  99.74944234]), 'score_time': array([0.00299072, 0.00398874, 0.0029912 , 0.00399351, 0.0039866 ,
       0.00196743, 0.00199723, 0.0029912 , 0.0029397 , 0.00295162]), 'test_score': array([-37427.00684932, -35948.49657534, -33200.3390411 , -35461.03424658,
       -33283.60273973, -36594.01369863, -34512.54794521, -34966.42808219,
       -33645.8390411 , -35097.64041096]), 'train_score': array([ -80.05136986,  -37.67123288,  -20.11986301,  -95.03424658,
        -83.98972603, -131.93493151, -117.80821918,  -41.5239726 ,
       -150.34246575, -123.37328767])}


D:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Running the classification on GaussianNB
{'fit_time': array([0.03690124, 0.0359478 , 0.03691053, 0.03195357, 0.0309546 ,
       0.02992535, 0.03490663, 0.02992034, 0.03092384, 0.03391838]), 'score_time': array([0.11170459, 0.10667133, 0.10175896, 0.09570765, 0.08972287,
       0.08776665, 0.08975863, 0.08781481, 0.09070492, 0.10167885]), 'test_score': array([-41539.9109589 , -41873.15068493, -40002.26369863, -41550.62671233,
       -43314.26712329, -42385.71917808, -39219.47945205, -37619.25342466,
       -37468.32191781, -35620.18150685]), 'train_score': array([-4073.97260274, -3383.90410959, -4144.43493151, -4047.60273973,
       -4353.16780822, -4487.32876712, -3874.82876712, -3873.8869863 ,
       -4671.40410959, -4365.06849315])}
Running the classification on GradientBoostingRegressor
{'fit_time': array([0.72506642, 0.70809221, 0.72110415, 0.72505021, 0.73902297,
       0.69810724, 0.70615149, 0.70412803, 0.70810676, 0.70516443]), 'score_time': array([0.00197792, 0.00099683, 0.000

,MLA Name,MLA Parameters,MLA Train Error Mean,MLA Test Error Mean,MLA Test Error 3*STD,MLA Time
3,GradientBoostingRegressor,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ...",-10144.6,-16514.1,2872.73,0.713999
5,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",0,-17256.3,2962.77,1.99937
8,RandomForestRegressor,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri...",-6648.03,-17987.9,2965.8,1.99409
4,BaggingRegressor,"{'base_estimator': None, 'bootstrap': True, 'b...",-7738.55,-19325.8,3435.11,0.245746
6,DecisionTreeRegressor,"{'ccp_alpha': 0.0, 'criterion': 'mse', 'max_de...",0,-26506.8,2371.7,0.0398882
7,KNeighborsRegressor,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",-20602,-27640.5,5001.99,0.0336199
1,LogisticRegression,"{'C': 1.0, 'class_weight': None, 'dual': False...",-88.1849,-35013.7,4003.44,97.2047
2,GaussianNB,"{'priors': None, 'var_smoothing': 1e-09}",-4127.56,-40059.3,7149.49,0.0332262
0,LinearRegression,"{'copy_X': True, 'fit_intercept': True, 'n_job...",-16447.3,-2.8595e+13,1.38156e+14,0.0100727


In [139]:
model = GradientBoostingRegressor(n_estimators=400)
model.fit(X_train, Y_train)
preds_train = model.predict(X_train)
preds_test = model.predict(X_test)

result = pd.DataFrame({
    'Id':X_Id,
    'SalePrice':preds_test
})

result.to_csv("dataset/result_gb.csv",index=False)

In [123]:
X_trainn, X_valid, Y_trainn, Y_valid = train_test_split(X_train, Y_train)

In [140]:
def get_mae_valid(model,X_train, X_valid, Y_train, Y_valid):
    model.fit(X_train, Y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(Y_valid, preds)

print(get_mae_valid(model,X_trainn, X_valid, Y_trainn, Y_valid))

15653.447190680452


In [119]:
# TODO:
#     1. Polynomial features
#     2. Grid Search
#     3. Improve ordinal variables
#     4. Feature Selection
#     5. Remove outliers

In [121]:
GradientBoostingRegressor?